In [1]:
from datasets import load_dataset, load_from_disk, concatenate_datasets
import re

#加载数据集
dataset = load_dataset('imdb')
#dataset = load_from_disk('datas/imdb')
dataset = concatenate_datasets(list(dataset.values()))


def f(data):
    text = []
    for i in data['text']:
        #移除br
        i = i.lower().replace('<br />', '')

        #移除标点符号
        i = re.sub('\W', ' ', i)

        #合并连续的空格
        i = re.sub('\s{2,}', ' ', i)

        #替换数字
        i = re.sub('\d+', '<NUM>', i)

        #切词
        i = i.strip().split(' ')

        text.append(i)

    return {'text': text}


dataset = dataset.map(f,
                      batched=True,
                      batch_size=1000,
                      num_proc=4,
                      remove_columns=['label'])


#过滤长度
def f(data):
    return [len(set(i)) > 12 + 6 + 1 for i in data['text']]


dataset = dataset.filter(f, batched=True, batch_size=1000, num_proc=4)

len(dataset), dataset[0]

Loading cached processed dataset at datas/imdb/train/cache-5a2574d94930e04f.arrow
Loading cached processed dataset at datas/imdb/unsupervised/cache-64b35eaa8538105d.arrow
Loading cached processed dataset at datas/imdb/test/cache-2842833e178d5d6d.arrow


Loading cached processed dataset at datas/imdb/unsupervised/cache-e4909fbbc588c3a2.arrow


Loading cached processed dataset at datas/imdb/train/cache-38b4085fabf18400_00000_of_00004.arrow


Loading cached processed dataset at datas/imdb/test/cache-38b4085fabf18400_00001_of_00004.arrow


Loading cached processed dataset at datas/imdb/unsupervised/cache-38b4085fabf18400_00002_of_00004.arrow


Loading cached processed dataset at datas/imdb/unsupervised/cache-38b4085fabf18400_00003_of_00004.arrow


(99918,
 {'text': ['i',
   'rented',
   'i',
   'am',
   'curious',
   'yellow',
   'from',
   'my',
   'video',
   'store',
   'because',
   'of',
   'all',
   'the',
   'controversy',
   'that',
   'surrounded',
   'it',
   'when',
   'it',
   'was',
   'first',
   'released',
   'in',
   '<NUM>',
   'i',
   'also',
   'heard',
   'that',
   'at',
   'first',
   'it',
   'was',
   'seized',
   'by',
   'u',
   's',
   'customs',
   'if',
   'it',
   'ever',
   'tried',
   'to',
   'enter',
   'this',
   'country',
   'therefore',
   'being',
   'a',
   'fan',
   'of',
   'films',
   'considered',
   'controversial',
   'i',
   'really',
   'had',
   'to',
   'see',
   'this',
   'for',
   'myself',
   'the',
   'plot',
   'is',
   'centered',
   'around',
   'a',
   'young',
   'swedish',
   'drama',
   'student',
   'named',
   'lena',
   'who',
   'wants',
   'to',
   'learn',
   'everything',
   'she',
   'can',
   'about',
   'life',
   'in',
   'particular',
   'she',
   'wants'

In [2]:
import torch


#构建字典
def get_vocab():
    #统计词频
    vocab = {}
    for i in range(len(dataset)):
        sent = dataset[i]['text']
        for word in sent:
            if word not in vocab:
                vocab[word] = len(vocab)

        if i % 10000 == 0:
            print(i, len(vocab))

    return vocab


vocab = get_vocab()

len(vocab), vocab['girl']

0 177
10000 49100
20000 67857
30000 81043
40000 92366
50000 102680
60000 111888
70000 120832
80000 128757
90000 136247


(143665, 1275)

In [3]:
#编码
def f(data):
    sents = []
    for sent in data['text']:
        sent = [vocab[word] for word in sent]
        sents.append(sent)

    return {'text': sents}


dataset = dataset.map(f, batched=True, batch_size=1000, num_proc=4)

#转list
dataset = dataset['text']

len(dataset), dataset[0]

Loading cached processed dataset at datas/imdb/train/cache-34f7a7ecafbce232.arrow


Loading cached processed dataset at datas/imdb/train/cache-8c2a38b799e372d4.arrow
Loading cached processed dataset at datas/imdb/train/cache-243d1634b6a88d12.arrow


Loading cached processed dataset at datas/imdb/train/cache-02ddcb1d4dc0ee62.arrow


(99918,
 [0,
  1,
  0,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  16,
  18,
  19,
  20,
  21,
  22,
  0,
  23,
  24,
  14,
  25,
  19,
  16,
  18,
  26,
  27,
  28,
  29,
  30,
  31,
  16,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  10,
  42,
  43,
  44,
  0,
  45,
  46,
  34,
  47,
  36,
  48,
  49,
  12,
  50,
  51,
  52,
  53,
  40,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  34,
  62,
  63,
  64,
  65,
  66,
  67,
  21,
  68,
  64,
  61,
  34,
  69,
  70,
  71,
  34,
  72,
  73,
  74,
  10,
  75,
  76,
  77,
  12,
  78,
  79,
  80,
  66,
  81,
  82,
  83,
  84,
  85,
  12,
  86,
  87,
  88,
  89,
  83,
  21,
  12,
  90,
  91,
  21,
  92,
  93,
  94,
  88,
  95,
  96,
  10,
  97,
  66,
  98,
  99,
  76,
  100,
  64,
  101,
  102,
  103,
  70,
  56,
  104,
  105,
  88,
  106,
  107,
  77,
  108,
  109,
  66,
  0,
  2,
  3,
  4,
  51,
  14,
  22,
  110,
  111,
  36,
  18,
  43,
  112,
  45,
  12,
  102,
  88,
  113

In [4]:
import random


def get_data():
    text = random.choice(dataset)

    i = random.randint(3, len(text) - 4)
    x = text[i]

    pos = text[i - 3:i + 4]
    pos.pop(3)

    neg = []
    while len(neg) != 12:
        neg = random.sample(text, 12 + 6)
        neg = [i for i in neg if i not in pos]
        neg = neg[:12]

    return x, pos, neg


get_data()

(747,
 [1092, 887, 303, 48, 40, 2110],
 [51, 10, 22, 161, 51, 211, 182, 5497, 12, 12, 1764, 528])

In [5]:
import torch


def get_batch():
    x = []
    pos = []
    neg = []
    for _ in range(128):
        data = get_data()
        x.append(data[0])
        pos.append(data[1])
        neg.append(data[2])

    x = torch.LongTensor(x)
    pos = torch.LongTensor(pos)
    neg = torch.LongTensor(neg)

    return x, pos, neg


get_batch()

(tensor([  567,  7554,   366,   506,   390,    10,    70,    40,  4023,     6,
           625,    47,  3047,    11,   176,  9379,    60,   646,    88,  1861,
            22,   103,  2937,   158,     2,   390,   225,   832, 90144,   182,
            12,    53,    16,    12, 24811,    25,   971,   201,  1275,   616,
          3033,  6754,     6,   120,  4307,   341,  1007,    14,    21,    10,
         15000,    36,  2108,    36,    51,  1632,  1499,  1035,    51,  1483,
           366,  3857,    27,    21, 16537,   304,  1046,   103,  2588, 16838,
           268,   175,   122,    88,   696,  1571, 25813,    21,   304,   409,
           156,  2216,    12,    12,   456,    65,    40,    16,   673,   393,
            12,  2284,   929,  1092, 36018,    62,  5269,   772,    66,    29,
           175,    12,   191,    70,    51, 41481,  5620, 31047,   215,   115,
           618,   191,  5594,  1351,   618,     0,   316,   155,    88, 11769,
           555,   116,    50,  1092,    76,   120,  

In [6]:
def get_cos_loss(x, pos, neg):
    #x -> [8, 150]
    #pos -> [8, 6, 150]
    #neg -> [8, 12, 150]

    #[8, 150] -> [8, 150, 1]
    x = x.unsqueeze(dim=2)

    #[8, 6, 150],[8, 150, 1] -> [8, 6]
    loss_pos = torch.bmm(pos, x).squeeze(dim=2)

    #[8, 12, 150],[8, 150, 1] -> [8, 12]
    loss_neg = torch.bmm(neg, -x).squeeze(dim=2)

    #[8, 6] -> [8]
    loss_pos = loss_pos.sigmoid().clip(min=1e-8).log().sum(dim=1)

    #[8, 12] -> [8]
    loss_neg = loss_neg.sigmoid().clip(min=1e-8).log().sum(dim=1)

    return -(loss_pos + loss_neg).mean()


a, b, c = torch.randn(8, 150), torch.randn(8, 6, 150), torch.randn(8, 12, 150)
get_cos_loss(a, b, c)

tensor(55.8218)

In [7]:
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()

        #V是词汇量
        self.embed = torch.nn.Embedding(num_embeddings=len(vocab),
                                        embedding_dim=150)

        #初始化参数
        self.embed.weight.data.uniform_(-0.01, 0.01)

    def forward(self, x, pos, neg):
        #编码
        #[8] -> [8, 2]
        x = self.embed(x)

        #[8, 6] -> [8, 6, 2]
        pos = self.embed(pos)

        #[8, 12] -> [8, 12, 2]
        neg = self.embed(neg)

        return get_cos_loss(x, pos, neg)


model = Model()

model(*get_batch())

tensor(12.4770, grad_fn=<NegBackward0>)

In [8]:
def test(test_words):
    embed = model.embed.weight.data.clone()
    vocab_keys = list(vocab.keys())

    for word in test_words:
        x = embed[vocab[word]]
        score = torch.nn.functional.cosine_similarity(x, embed)
        topk = score.topk(k=5).indices
        topk = [vocab_keys[k] for k in topk]
        print(word, topk)


test(['girl', 'bus', 'green', 'doctor', 'dog', 'queen', 'italy'])

girl ['girl', 'lothar', 'biographic', 'sylke', '<NUM>cs']
bus ['bus', 'heavyweights', 'quintuples', 'vine', 'steacy']
green ['green', 'chiba', 'putzi', 'shield', 'charectar']
doctor ['doctor', 'kf', 'friesian', 'maaaaaaaaaa', 'klumholtz']
dog ['dog', 'dries', 'interviews', 'pirouette', 'ishimoto']
queen ['queen', 'halburton', 'roseane', 'swng', 'technologists']
italy ['italy', 'hamdan', 'mobility', 'malamud', 'july']


In [9]:
def train():
    global model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)

    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_sum = 0
    for epoch in range(2000001):
        batch = get_batch()
        batch = [i.to(device) for i in batch]

        loss = model(*batch)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        loss_sum += loss.item()

        if epoch % 10000 == 0:
            print(epoch, loss_sum)
            test(['girl', 'bus', 'green', 'doctor', 'dog', 'queen', 'italy'])
            loss_sum = 0

        if epoch % 100000 == 0:
            torch.save(model.cpu(), 'models/imdb_%d.model' % epoch)
            model = model.to(device)

    model = model.cpu()


train()

0 12.477056503295898
girl ['girl', 'lothar', 'biographic', 'sylke', '<NUM>cs']
bus ['bus', 'heavyweights', 'quintuples', 'vine', 'steacy']
green ['green', 'chiba', 'putzi', 'shield', 'charectar']
doctor ['doctor', 'kf', 'urning', 'maaaaaaaaaa', 'plaintiffs']
dog ['dog', 'interviews', 'dries', 'routemaster', 'channeled']
queen ['queen', 'halburton', 'exorcisms', 'swng', 'roseane']
italy ['italy', 'hamdan', 'mobility', 'malamud', 'july']
10000 123871.56939983368
girl ['girl', 'boy', 'guy', 'woman', 'man']
bus ['bus', 'misconceptions', 'shelter', 'drag', 'chairs']
green ['green', 'makeup', 'klapisch', 'camera', 'piano']
doctor ['doctor', 'hanson', 'projected', 'joins', 'carrey']
dog ['dog', 'precense', 'overstaying', 'chair', 'stiffness']
queen ['queen', 'fag', 'fey', 'army', 'taming']
italy ['italy', 'whets', 'customary', 'eyes', 'keeping']
20000 123611.64912414551
girl ['girl', 'woman', 'boy', 'man', 'meets']
bus ['bus', 'truck', 'projected', 'egged', 'gila']
green ['green', 'thoughnoth

200000 123672.05637645721
girl ['girl', 'boy', 'young', 'woman', 'man']
bus ['bus', 'handrail', 'skydive', 'vacation', 'stipulates']
green ['green', 'sipsey', 'chrosite', 'gables', 'red']
doctor ['doctor', 'sees', 'dráuzio', 'bouchitey', 'filmmaker']
dog ['dog', 'scraggly', 'festering', 'reasosn', 'gottdog']
queen ['queen', 'ragneks', 'flamholc', 'ice', 'bride']
italy ['italy', 'larien', 'norther', 'england', 'yamika']
210000 123701.95130634308
girl ['girl', 'boy', 'kid', 'meets', 'woman']
bus ['bus', 'handrail', 'skydive', 'rehert', 'driver']
green ['green', 'gables', 'seth', 'thoughnothing', 'chrosite']
doctor ['doctor', 'scientist', 'skulks', 'guy', 'moonlights']
dog ['dog', 'cadabra', 'wag', 'nonanimated', 'pekingnese']
queen ['queen', 'latifah', 'tropicana', 'quigley', 'oomphale']
italy ['italy', 'loowows', 'england', 'basset', 'mexico']
220000 123711.49648189545
girl ['girl', 'boy', 'young', 'woman', 'blind']
bus ['bus', 'train', 'truck', 'toilette', 'swath']
green ['green', 'red

400000 123840.44580459595
girl ['girl', 'boy', 'adapatable', 'man', 'woman']
bus ['bus', 'truck', 'journey', 'train', 'cableor']
green ['green', 'soylent', 'red', 'sliiiiimmmmmme', 'sipsey']
doctor ['doctor', 'scientist', 'apprentice', 'journalist', 'kinickie']
dog ['dog', 'dockers', 'leadbetter', 'yoshitaka', 'nails']
queen ['queen', 'superthug', 'bee', 'oomphale', 'latifah']
italy ['italy', 'japan', 'janetsteps', 'germany', 'england']
410000 123837.3357000351
girl ['girl', 'boy', 'woman', 'young', 'hatcheck']
bus ['bus', 'succ', 'guide', 'truck', 'licoln']
green ['green', 'seth', 'sliiiiimmmmmme', 'sevalas', 'sipsey']
doctor ['doctor', 'scientist', 'plays', 'csu', 'kid']
dog ['dog', 'vecino', 'silveira', 'nails', 'girl']
queen ['queen', 'bee', 'superthug', 'scream', 'latifah']
italy ['italy', 'japan', 'england', 'germany', 'in']
420000 123856.29262828827
girl ['girl', 'boy', 'woman', 'young', 'tianka']
bus ['bus', 'turning', 'wisconsite', 'plainville', 'nudist']
green ['green', 'seth

600000 123924.04013061523
girl ['girl', 'boy', 'young', 'woman', 'kid']
bus ['bus', 'handrail', 'skankville', 'ride', 'vaccarro']
green ['green', 'seth', 'blue', 'purple', 'red']
doctor ['doctor', 'scientist', 'photographer', 'mysterious', 'aliesn']
dog ['dog', 'giant', 'peek', 'saknussemm', 'puppy']
queen ['queen', 'latifah', 'bee', 'oomphale', 'scream']
italy ['italy', 'spain', 'in', 'almeria', 'japan']
610000 123918.12096786499
girl ['girl', 'boy', 'kid', 'young', 'hatcheck']
bus ['bus', 'driver', 'car', 'ride', 'train']
green ['green', 'blue', 'seth', 'red', 'gables']
doctor ['doctor', 'scientist', 'bromilow', 'owned', 'nokitofa']
dog ['dog', 'bim', 'leadbetter', 'cat', 'shark']
queen ['queen', 'bee', 'shannon', 'latifah', 'superthug']
italy ['italy', 'england', 'in', 'spain', 'sweatlodge']
620000 123947.68498039246
girl ['girl', 'boy', 'young', 'woman', 'hatcheck']
bus ['bus', 'truck', 'moates', 'crashes', 'puzzel']
green ['green', 'seth', 'sliiiiimmmmmme', 'blue', 'red']
doctor [

800000 124078.02312088013
girl ['girl', 'boy', 'woman', 'khrysikou', 'young']
bus ['bus', 'train', 'shelf', 'vacation', 'plane']
green ['green', 'sliiiiimmmmmme', 'seth', 'blue', 'donaldsutherland']
doctor ['doctor', 'scientist', 'kid', 'likes', 'who']
dog ['dog', 'justa', 'pigsty', 'savageis', 'knife']
queen ['queen', 'scream', 'rekay', 'gorilla', 'roadcrew']
italy ['italy', 'england', 'dolomites', 'winnepeg', 'early']
810000 124077.27696609497
girl ['girl', 'boy', 'woman', 'young', 'enthrallingly']
bus ['bus', 'ship', 'horse', 'plane', 'toy']
green ['green', 'blue', 'sliiiiimmmmmme', 'seth', 'techninques']
doctor ['doctor', 'violeta', 'guy', 'rivault', 'who']
dog ['dog', 'dealer', 'big', 'bicycle', 'train']
queen ['queen', 'bee', 'scream', 'ellery', 'kressley']
italy ['italy', 'england', 'in', 'exchang', 'stanywck']
820000 124097.04174232483
girl ['girl', 'boy', 'woman', 'young', 'kid']
bus ['bus', 'plane', 'island', 'driver', 'zdf']
green ['green', 'sliiiiimmmmmme', 'seth', 'blue', 

1000000 124266.33344364166
girl ['girl', 'boy', 'young', 'woman', 'kid']
bus ['bus', 'train', 'truck', 'driver', 'roof']
green ['green', 'techninques', 'blue', 'red', 'seth']
doctor ['doctor', 'scientist', 'macdormand', 'who', 'boy']
dog ['dog', 'leadbetter', 'wampus', 'drag', 'girl']
queen ['queen', 'cordaraby', 'queens', 'bride', 'lady']
italy ['italy', 'england', 'in', 'spain', 'subserviant']
1010000 124263.91089916229
girl ['girl', 'boy', 'young', 'kid', 'woman']
bus ['bus', 'station', 'force', 'tour', 'ride']
green ['green', 'seth', 'blue', 'scissorhand', 'red']
doctor ['doctor', 'deranged', 'guy', 'scientist', 'puttering']
dog ['dog', 'stray', 'joke', 'cat', 'vampire']
queen ['queen', 'kressley', 'posher', 'latifah', 'capet']
italy ['italy', 'mexico', 'spain', 'in', 'sovie']
1020000 124284.85352706909
girl ['girl', 'boy', 'kid', 'young', 'woman']
bus ['bus', 'sitting', 'dive', 'brittania', 'merengue']
green ['green', 'seth', 'techninques', 'orange', 'blue']
doctor ['doctor', 'sci

1210000 124470.88416576385
girl ['girl', 'boy', 'young', 'woman', 'kid']
bus ['bus', 'crash', 'taxi', 'driver', 'lenthy']
green ['green', 'rose', 'seth', 'knight', 'scissorhand']
doctor ['doctor', 'everyman', 'who', 'bumbling', 'scientist']
dog ['dog', 'bites', 'stray', 'consolidations', 'cranby']
queen ['queen', 'kressley', 'highlander', 'admira', 'alien']
italy ['italy', 'in', 'france', 'russia', 'japan']
1220000 124457.28697490692
girl ['girl', 'boy', 'young', 'kid', 'woman']
bus ['bus', 'crash', 'plane', 'road', 'driver']
green ['green', 'seth', 'techninques', 'stathom', 'sliiiiimmmmmme']
doctor ['doctor', 'scientist', 'nokitofa', 'plays', 'who']
dog ['dog', 'giant', 'stray', 'fudoh', 'sweaty']
queen ['queen', 'kressley', 'alien', 'latifah', 'enemy']
italy ['italy', 'in', 'europe', 'winnepeg', 'peidmont']
1230000 124472.55234718323
girl ['girl', 'boy', 'young', 'meets', 'brisebois']
bus ['bus', 'plane', 'clouds', 'road', 'taxi']
green ['green', 'techninques', 'seth', 'blue', 'sliii

1410000 124660.96550273895
girl ['girl', 'boy', 'young', 'kid', 'named']
bus ['bus', 'crash', 'plane', 'driver', 'on']
green ['green', 'techninques', 'red', 'blue', 'stathom']
doctor ['doctor', 'beauhunk', 'terrorist', 'darthvader', 'macdormand']
dog ['dog', 'stray', 'pet', 'giant', 'lunch']
queen ['queen', 'scream', 'abdicates', 'bee', 'nisei']
italy ['italy', 'germany', 'in', 'patriarchial', 'parrotosaurus']
1420000 124674.20229148865
girl ['girl', 'boy', 'kid', 'young', 'woman']
bus ['bus', 'driver', 'succ', 'whaaaaaaa', 'subway']
green ['green', 'red', 'techninques', 'bright', 'light']
doctor ['doctor', 'scientist', 'merrilywhen', 'cop', 'denosed']
dog ['dog', 'stray', 'pet', 'siska', 'busines']
queen ['queen', 'scream', 'spider', 'nisei', 'kressley']
italy ['italy', 'parrotosaurus', 'in', 'overdecorated', 'nordish']
1430000 124636.46362876892
girl ['girl', 'boy', 'young', 'kid', 'meets']
bus ['bus', 'driver', 'train', 'road', 'lunch']
green ['green', 'blue', 'red', 'techninques', 

1610000 124867.63029766083
girl ['girl', 'boy', 'young', 'woman', 'meets']
bus ['bus', 'plane', 'train', 'rainy', 'truck']
green ['green', 'seth', 'paintwork', 'techninques', 'orange']
doctor ['doctor', 'caligary', 'dipsomaniacal', 'macdormand', 'denosed']
dog ['dog', 'consolidations', 'stray', 'size', 'leadbetter']
queen ['queen', 'bee', 'posher', 'tysack', 'warrior']
italy ['italy', 'in', 'france', 'punta', 'mid']
1620000 124873.67226409912
girl ['girl', 'boy', 'young', 'rich', 'brisebois']
bus ['bus', 'plane', 'boat', 'train', 'driver']
green ['green', 'seth', 'red', 'blue', 'orange']
doctor ['doctor', 'nerd', 'girl', 'cousin', 'plays']
dog ['dog', 'gottdog', 'bus', 'cranby', 'stray']
queen ['queen', 'scream', 'queens', 'bee', 'phoenicia']
italy ['italy', 'france', 'in', 'ahu', 'dionysus']
1630000 124909.52609729767
girl ['girl', 'boy', 'young', 'woman', 'kid']
bus ['bus', 'train', 'station', 'plane', 'crash']
green ['green', 'techninques', 'blue', 'seth', 'paintwork']
doctor ['doct

1820000 125038.54052448273
girl ['girl', 'boy', 'woman', 'doutey', 'blond']
bus ['bus', 'train', 'toy', 'driver', 'taxi']
green ['green', 'seth', 'red', 'paintwork', 'sliiiiimmmmmme']
doctor ['doctor', 'caligary', 'luisinho', 'anatomist', 'chilcot']
dog ['dog', 'shaggy', 'sin', 'mutant', 'tunnelful']
queen ['queen', 'bee', 'quarterback', 'scream', 'admira']
italy ['italy', 'england', 'rockaway', 'japan', 'rome']
1830000 125080.90907859802
girl ['girl', 'boy', 'young', 'woman', 'who']
bus ['bus', 'driver', 'taxi', 'bat', 'ride']
green ['green', 'seth', 'paintwork', 'red', 'blue']
doctor ['doctor', 'scientist', 'crazed', 'played', 'graumann']
dog ['dog', 'stray', 'shaggy', 'doll', 'hot']
queen ['queen', 'bee', 'unto', 'admira', 'judging']
italy ['italy', 'seattle', 'france', 'rockaway', 'japan']
1840000 125084.71061325073
girl ['girl', 'boy', 'young', 'woman', 'doutey']
bus ['bus', 'truck', 'driver', 'taxi', 'train']
green ['green', 'paintwork', 'seth', 'red', 'blue']
doctor ['doctor', '

In [10]:
model = torch.load('models/imdb_2000000.model')

test(['girl', 'bus', 'green', 'doctor', 'dog', 'queen', 'italy'])

girl ['girl', 'boy', 'teenage', 'young', 'doutey']
bus ['bus', 'school', 'train', 'high', 'station']
green ['green', 'red', 'paintwork', 'blue', 'scissorhand']
doctor ['doctor', 'tough', 'noirishly', 'scientist', 'bromilow']
dog ['dog', 'stray', 'bull', 'juice', 'shop']
queen ['queen', 'queens', 'prom', 'roadcrew', 'scream']
italy ['italy', 'in', 'london', 'chicago', 'mid']
